# Test new versions of the code #

This notebook runs a new version of trisurf through the three modes: from tape, from binary, and from vtu.

**This notebook does not re-`make` the trisurf: The test is on the existing trisurf at the location!**

In [1]:
import subprocess, shutil, datetime
from pathlib import Path

import numpy as np
import pandas as pd

from vtu import PyVtu
from ts_auto_wrapper import TSWrapper

Create the wrapper for the new version. The path need to be supplied, but here is a nice guess

In [2]:
path_to_trisurf = Path("../cluster-trisurf")
if not path_to_trisurf.exists():
    path_to_trisurf = Path("../../cluster-trisurf")
    if not path_to_trisurf.exists():
        path_to_trisurf = Path("~/cluster_trisurf")
path_to_trisurf = path_to_trisurf.resolve()


ts=TSWrapper(path_to_trisurf)
# we could also use older versions
# ts=TSWrapper(Path("/opt/workspace/msc_project/trisurf_samo/trisurf-ng/"))

trisurf_test = (Path('.')/"trisurf_test").resolve()

Here we supply where and how to run the test.  
This essentially recreate  
* creating a folder  
* running the current trisurf version with tape and command line agruments `extra args`  
* collect the statistics ad output for each phase  
* delete the timesteps  

The **main** reason is to make sure the trisurf version even works.

In [3]:
test_folder=None # name of the folder for the test
tape=None # tape file location
extra_args = [] # comand line arguments
destroy_timesteps_at_end=True # destroy all timesteps after the test
timeout = 750 # seconds to timeout for each `trisurf` test, fit for samo trisurf with nshell=17

In [4]:
# test_folder = trisurf_test/'2023_05_29_main_egg_carton'
# # test_folder = trisurf_test/'2023_05_29_anisotropy_samo_version'
# extra_args = ['-c adhesion_model=1,adhesion_geometry=6,adhesion_scale=2']
# destroy_timesteps_at_end=False # destroy all timesteps after the test
# timeout = 120 # seconds to timeout for each `trisurf` test
# timeout = 750

In [5]:
# test_folder=trisurf_test/'2023_06_24_unalloc1'
# tape=None
# extra_args = []
# destroy_timesteps_at_end=True # destroy all timesteps after the test
# timeout = 750 # seconds to timeout for each `trisurf` test, fit for samo trisurf with nshell=17

### Make a new folder if there isn't one ###

In [6]:
new_folder=None
if not test_folder:
    # make up some appropriate folder
    now = datetime.datetime.now()
    ret = subprocess.run(['git', 'status'],cwd=ts.path_to_trisurf,capture_output=True)
    branch = ret.stdout.decode().partition('On branch ')[-1].splitlines()[0].strip()
    new_folder = f'{now.year:04}_{now.month:02}_{now.day:02}_{branch}'
    test_folder = trisurf_test/new_folder
new_folder

'2023_07_02_main'

In [7]:
if not test_folder.exists():
    test_folder.mkdir()
    test_folder.joinpath('test_results').mkdir()
elif new_folder is not None:
    raise ValueError(f'folder {test_folder} for today already exists: Rename or delete to run test')

### copy the tape to the new folder ###

In [8]:
if tape is None:
    tape=ts.path_to_trisurf/"src/tape"
shutil.copy(tape,test_folder/"tape")

PosixPath('/home/toriy/yoav/trisurf-python/trisurf_test/2023_07_02_main/tape')

## Check initizliaztion from tape ##

The subprocess is equivalent to the command  
`trisuf --force-from-tape extra_args[0] extra_args[1]...`  
running in the testfolder

In [9]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf", "--force-from-tape", *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

Print the output and save it to the test results.

In [10]:
print(ret.stdout.decode())
with open(test_folder/"test_results"/"force_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode()+'\n'+ret.stderr.decode())
shutil.copy(test_folder/"statistics.csv",test_folder/"test_results"/"force_statistics.csv")

[2023-07-02 00:23:39] TRISURF-NG v. 13597e6, compiled on: Jul  1 2023 02:04:58.
[2023-07-02 00:23:39] Programming done by: Samo Penic and Miha Fosnaric
[2023-07-02 00:23:39] Released under terms of GPLv3
[2023-07-02 00:23:39] Starting program...

[2023-07-02 00:23:39] ************************************************
[2023-07-02 00:23:39] **** Generating initial geometry from tape *****
[2023-07-02 00:23:39] ************************************************

[2023-07-02 00:23:39] Starting initial_distribution on vesicle with 10 shells!...
[2023-07-02 00:23:40] initial_distribution finished!
[2023-07-02 00:23:40] simulation seed 1688246620
[2023-07-02 00:23:40] Setting volume V0=1041.99378875998718286
[2023-07-02 00:23:40] Setting area A0=623.53829072480641571
[2023-07-02 00:23:48] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:23:59] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:24:11] Done 3 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:24:18] 

PosixPath('/home/toriy/yoav/trisurf-python/trisurf_test/2023_07_02_main/test_results/force_statistics.csv')

## Check start from binary dump ##

In [11]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf","--reset-iteration-count", *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [12]:
print(ret.stdout.decode())
with open(test_folder/"test_results"/"reset_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode()+'\n'+ret.stderr.decode())
shutil.copy(test_folder/"statistics.csv",test_folder/"test_results"/"reset_statistics.csv")

[2023-07-02 00:25:05] TRISURF-NG v. 13597e6, compiled on: Jul  1 2023 02:04:58.
[2023-07-02 00:25:05] Programming done by: Samo Penic and Miha Fosnaric
[2023-07-02 00:25:05] Released under terms of GPLv3
[2023-07-02 00:25:05] Starting program...

[2023-07-02 00:25:05] **********************************************************************
[2023-07-02 00:25:05] **** Recreating vesicle from dump file and continuing simulation *****
[2023-07-02 00:25:05] **********************************************************************

[2023-07-02 00:25:06] simulation seed 1688246706
[2023-07-02 00:25:06] Setting volume V0=1669.94225308700856658
[2023-07-02 00:25:06] Setting area A0=705.37220761160756410
[2023-07-02 00:25:16] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:25:23] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:25:32] Done 3 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:25:41] Done 4 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:25:49]

PosixPath('/home/toriy/yoav/trisurf-python/trisurf_test/2023_07_02_main/test_results/reset_statistics.csv')

## Check start from vtu ##

In [13]:
test_folder.joinpath('.status').unlink() # remove status so it resets iterations

In [14]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf",
                      "--restore-from-vtk",'timestep_000000.vtu', *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [15]:
print(ret.stdout.decode())
with open(test_folder/"test_results"/"restore_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode()+'\n'+ret.stderr.decode())
shutil.copy(test_folder/"statistics.csv",test_folder/"test_results"/"restore_statistics.csv")

[2023-07-02 00:26:26] TRISURF-NG v. 13597e6, compiled on: Jul  1 2023 02:04:58.
[2023-07-02 00:26:26] Programming done by: Samo Penic and Miha Fosnaric
[2023-07-02 00:26:26] Released under terms of GPLv3
[2023-07-02 00:26:26] Starting program...

[2023-07-02 00:26:26] ************************************************
[2023-07-02 00:26:26] **** Restoring vesicle from VTK points list ****
[2023-07-02 00:26:26] ************************************************

[2023-07-02 00:26:26] No .status file. The iteration count will start from 0
[2023-07-02 00:26:26] simulation seed 1688246786
[2023-07-02 00:26:26] Setting volume V0=1656.13771250929357848
[2023-07-02 00:26:26] Setting area A0=697.43564016036123121
[2023-07-02 00:26:35] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:26:42] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:26:52] Done 3 out of 10 iterations (x 1000 MC sweeps).
[2023-07-02 00:27:01] Done 4 out of 10 iterations (x 1000 MC sweeps).
[2023-07-0

PosixPath('/home/toriy/yoav/trisurf-python/trisurf_test/2023_07_02_main/test_results/restore_statistics.csv')

### **Files will be cleaned up at the end of the script!** ###

## More tests ##

Load last file as both PyVtu and vesicle. See what is the difference

In [16]:
try:
    fileloc = list(sorted(test_folder.glob('timestep*')))[-1]
except NameError:
    test_folder = list(sorted(trisurf_test.glob('./[!.]*')))[-1]
    fileloc = list(sorted(test_folder.glob('timestep*')))[-1]
fileloc

PosixPath('/home/toriy/yoav/trisurf-python/trisurf_test/2023_07_02_main/timestep_000009.vtu')

In [17]:
vesicle = ts.parseDump(fileloc)

In [18]:
v = PyVtu(fileloc)
v2 = PyVtu(fileloc) # we will place here the values from vesicle

Find the fields of the vesicle

In [19]:
vertex_fields = [x[0] for x in ts.ts_vertex._fields_]
vertex_fields

['x',
 'y',
 'z',
 'mean_curvature',
 'gaussian_curvature',
 'mean_energy',
 'gaussian_energy',
 'energy',
 'xk',
 'xk2',
 'w',
 'c',
 'nx',
 'ny',
 'nz',
 'nx2',
 'ny2',
 'nz2',
 'f',
 'fx',
 'fy',
 'fz',
 'ad_w',
 'd',
 'dx',
 'dy',
 'dz',
 'eig0',
 'eig1',
 'eig2',
 'eig_v0',
 'eig_v1',
 'eig_v2',
 'area',
 'S',
 'mean_curvature2',
 'gaussian_curvature2',
 'mean_energy2',
 'gaussian_energy2',
 'neigh',
 'tristar',
 'bond',
 'cell',
 'grafted_poly',
 'cluster',
 'idx',
 'id',
 'neigh_no',
 'tristar_no',
 'bond_no',
 'type']

In [20]:
bond_fields = [x[0] for x in ts.ts_bond._fields_]
triangle_fields = [x[0] for x in ts.ts_triangle._fields_]
triangle_fields

['xnorm',
 'ynorm',
 'znorm',
 'xcirc',
 'ycirc',
 'zcirc',
 'area',
 'volume',
 'energy',
 'vertex',
 'neigh',
 'idx',
 'neigh_no']

Replace values from vesicle. This will change with versions!

In [21]:
def field_from(vesicle,field=None,lst='vlist'):
    try:
        return np.array([v.__getattribute__(field) for v in ts.iter_xlist(vesicle,lst)])
    except AttributeError:
        return None
def vector_field_from(vesicle,fields,lst='vlist'):
    try:
        return np.array([tuple(v.__getattribute__(field) for field in fields) for v in ts.iter_xlist(vesicle,lst)])
    except AttributeError:
        return None

def array_field_from(vesicle,field,lst='vlist'):
    try:
        return np.array([v.__getattribute__(field)[:] for v in ts.iter_xlist(vesicle,lst)])
    except AttributeError:
        return None

def replace_pyvtu_values_from_vesicle(v,vesicle):
    """Replace the values the PyVtu arrays with values from the wrapper."""
    blen = vesicle.contents.blist.contents.n
    tlen = vesicle.contents.tlist.contents.n
    v.add_array('pos',vector_field_from(vesicle,('x','y','z')))
    v.add_array('neigh_no',field_from(vesicle,'neigh_no'))
    v.add_array('c0',field_from(vesicle,'c'))
    v.add_array('e',field_from(vesicle,'energy'))
    # newer version stuff (can fail but who cares?)
    v.add_array('w',field_from(vesicle,'w'))
    v.add_array('f0',field_from(vesicle,'f'))
    v.add_array('normal',vector_field_from(vesicle,['nx','ny','nz']))
    v.add_array('force',vector_field_from(vesicle,['fx','fy','fz']))
    v.add_array('e',field_from(vesicle,'energy'))
    if 'xnorm' in triangle_fields:
        v.add_array('face_normal',np.concatenate((np.zeros((blen,3)),vector_field_from(vesicle,('xnorm','ynorm','znorm'),'tlist'))),parent='CellData')
    v.add_array('ad_w',field_from(vesicle,'ad_w'))
    if 'type' in vertex_fields:
        v.add_array('type', np.array([ts.byte_to_int(v.type) for v in ts.iter_xlist(vesicle)]))
    v.add_array('k',field_from(vesicle,'k'))
    v.add_array('k2',field_from(vesicle,'k2'))
    v.add_array('area',field_from(vesicle,'area'))
    v.add_array('gaussian_curvature',field_from(vesicle,'gaussian_curvature'))
    v.add_array('mean_curvature',field_from(vesicle,'mean_curvature'))
    v.add_array('gaussian_curvature',field_from(vesicle,'gaussian_curvature'))
    v.add_array('director',vector_field_from(vesicle,('dx','dy','dz')))
    v.add_array('mean_curvature2',field_from(vesicle,'mean_curvature2'))
    v.add_array('gaussian_curvature2',field_from(vesicle,'gaussian_curvature2'))
    v.add_array('eig0',array_field_from(vesicle,'eig0'))
    v.add_array('eig1',array_field_from(vesicle,'eig1'))
    v.add_array('eig2',array_field_from(vesicle,'eig2'))
    v.add_array('eigenvalue_0',field_from(vesicle,'eig_v0'))
    v.add_array('eigenvalue_1',field_from(vesicle,'eig_v1'))
    v.add_array('eigenvalue_2',field_from(vesicle,'eig_v2'))

    if 'xcirc' in triangle_fields:
        v.add_array('circumcenter',np.concatenate((np.zeros((blen,3)),vector_field_from(vesicle,('xcirc','ycirc','zcirc'),'tlist'))),parent='CellData')
    if 'S' in vertex_fields:
        v.add_array('dSd',np.array([x.S[0] for x in ts.iter_xlist(vesicle)]))
        v.add_array('dSt',np.array([x.S[1] for x in ts.iter_xlist(vesicle)]))
        v.add_array('tSd',np.array([x.S[2] for x in ts.iter_xlist(vesicle)]))
        v.add_array('tSt',np.array([x.S[3] for x in ts.iter_xlist(vesicle)]))
    v.add_array('tangent',np.cross(v.normal,v.director))
    
    v.add_array('mean_curvature3',v.mean_curvature+0)
    v.add_array('gaussian_curvature3',v.gaussian_curvature+0)
    v.add_array('normal3',v.normal+0)
    for i, vtx in enumerate(ts.iter_xlist(vesicle)):
        vtx_copy=ts.ts_vertex(**{field:vtx.__getattribute__(field) for field in vertex_fields})
        ts.tensor_curvature_energy2(vesicle,ts.pointer(vtx_copy))
        v.mean_curvature3[i]=vtx_copy.mean_curvature
        v.gaussian_curvature3[i]=vtx_copy.gaussian_curvature
        v.normal3[i]=vtx_copy.nx,vtx_copy.ny,vtx_copy.nz
    

In [22]:
try:
    replace_pyvtu_values_from_vesicle(v2,vesicle)
except AttributeError as e:
    print(f'partial replacement: {e}')

## Neighbor counts

In [23]:
_, counts = np.unique(v.blist, return_counts=True)
assert (counts==v2.neigh_no).all()

## Check that neighbors are ordered

They should be ordered, but the array might be circularly shifted i.e. `[0,1,2]==[2,0,1]` but not `[2,1,0]` or `[3,1,2]`

In [24]:
ordered=[]
ordered_identical=[]
for i in v.indices:
    vtx = vesicle.contents.vlist.contents.vtx[i].contents
    neighs = np.array([x.contents.idx for x in vtx.neigh[:vtx.neigh_no]])
    neighs_v = v.get_ordered_neighbors(i)
    ordered_identical.append((neighs==neighs_v).all())
    ordered.append(any([(neighs==np.roll(neighs_v,i)).all() for i, _ in enumerate(neighs) ]))

In [25]:
assert all(ordered)
all(ordered), all(ordered_identical) # should be (True,False)

(True, False)

### Make sure all values are close

In [26]:
close_arrays={k:np.allclose(v._arrays[k],v2._arrays[k]) for k in v._arrays}
close_arrays

{'pos': True,
 'blist': True,
 'tlist': True,
 'indices': True,
 'type': True,
 'c0': True,
 'w': True,
 'f0': True,
 'ad_w': True,
 'd0': True,
 'mean_curvature': True,
 'gaussian_curvature': True,
 'k': True,
 'k2': True,
 'mean_curvature2': True,
 'gaussian_curvature2': True,
 'area': True,
 'e': True,
 'mean_energy': True,
 'gaussian_energy': True,
 'normal': True,
 'force': False,
 'director': True,
 'eig0': True,
 'eig1': True,
 'bonding_energy': True,
 'face_normal': True}

#### check gaussian nonesense

In [27]:
if 'gaussian_curvature' in v._arrays:
    gdiff = v.gaussian_curvature-v.gaussian_curvature2
    print(f'gaussian curvature: max difference: {gdiff.max():.4e},'
          f' avg ± std: {gdiff.mean():.4e} ± {gdiff.std():.4e}')

gaussian curvature: max difference: 7.6328e-17, avg ± std: -3.7164e-19 ± 1.4914e-17


In [28]:
big_diffs = {k:(v._arrays[k]-v2._arrays[k])
             for k,val in close_arrays.items() if not val}
{k: f'max difference: {v.max():.4e}, avg ± std: {v.mean():.4e} ± {v.std():.4e}' for k, v in big_diffs.items()}

{'force': 'max difference: 9.9256e-02, avg ± std: 2.1294e-05 ± 6.0366e-03'}

In [29]:
vertex_dataframe=pd.DataFrame({k:val for k,val in v._arrays.items() if val.shape==v.indices.shape})
vertex_dataframe.describe()

,indices,type,c0,w,f0,ad_w,d0,mean_curvature,gaussian_curvature,k,k2,mean_curvature2,gaussian_curvature2,area,e,mean_energy,gaussian_energy
count,502.000000,502.000000,502.000000,502.000000,502.000000,502.0,502.000000,502.000000,502.000000,502.0,502.0,502.000000,502.000000,502.000000,502.000000,502.000000,502.000000
mean,250.500000,8.282869,0.049801,0.099602,0.099602,1.0,0.049801,0.156836,0.019183,20.0,-20.0,0.155768,0.019183,1.396246,1.219649,1.493271,-0.273622
std,145.059183,12.889971,0.149883,0.299767,0.299767,0.0,0.149883,0.112646,0.040222,0.0,0.0,0.111692,0.040222,0.138944,1.005075,1.667463,0.884232
min,0.000000,4.000000,0.000000,0.000000,0.000000,1.0,0.000000,-0.134036,-0.066169,20.0,-20.0,-0.134036,-0.066169,1.001258,0.008835,0.000002,-5.596283
25%,125.250000,4.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.076981,-0.005101,20.0,-20.0,0.076981,-0.005101,1.293072,0.482454,0.249919,-0.636588
50%,250.500000,4.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.148880,0.006694,20.0,-20.0,0.148563,0.006694,1.395669,0.950597,0.971284,-0.062699
75%,375.750000,4.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.228207,0.033135,20.0,-20.0,0.225948,0.033135,1.494857,1.686859,2.287298,0.226498
max,501.000000,47.000000,0.500000,1.000000,1.000000,1.0,0.500000,0.570085,0.233844,20.0,-20.0,0.570085,0.233844,1.826724,7.783330,13.379613,2.617151


#### Destory files

In [30]:
if destroy_timesteps_at_end:
    files_to_remove = list(test_folder.glob('timestep*'))
    excess_files = ('dump.bin','dout','output.pvd','statistics.csv')
    files_to_remove.extend(test_folder/x for x in excess_files)
    [file.unlink() for file in files_to_remove if file.exists()]